In [4]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
import pandas as pd

In [5]:
date = 20211109
list_url_base = "https://news.naver.com/main/list.naver"
list_url_start = f"?mode=LS2D&mid=shm&sid2=250&sid1=102&date={date}"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
}
https://news.naver.com/main/read.naver?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=102&sid2=250&oid=001&aid=0012794955
https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=102&sid2=250&oid=001&aid=0012794955

In [6]:
response = requests.get(list_url_base+list_url_start, headers=headers)
list_page = bs(response.text, "html.parser")

time.sleep(random.randrange(20, 30))


In [12]:
def get_read_content(read_url):
    response = requests.get(read_url, headers=headers)

    res = bs(response.text, "html.parser")
    journalist_name = res.find("div",{"class":"journalistcard_summary_name"}).text

    # likeit_list = res.find_all("span",{"class":"u_likeit_list_count _count"})
    # print(likeit_list)
    res = res.find("div", {"class": "_article_body_contents article_body_contents"})

    try:
        img_coment = res.find("em",{"class": "img_desc"}).text.strip()
    except:
        img_coment = ""

    full_con = res.text.strip().replace(img_coment,"").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}", "").replace("\n","")

    return img_coment, full_con, journalist_name


# get_read_content("https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=102&sid2=250&oid=469&aid=0000640122")\
#     .text.strip().replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n", "")


In [8]:

def get_content(news_list):
    news_page_df = None
    for news in news_list[4:5]:
        inf = {"date": None, "title": None, "lede": None,
            "writing": None,"journalist": None, "news_url": None, "img_url": None,
            "content": None, "img_coment": None}

        inf["date"] = date
        head = news.find("dl").find("dt")
        news_url = head.find("a")["href"]
        # cont = get_read_content(news_url)
        # inf["img_coment"] = cont[0]
        # inf["content"] = cont[1]
        # inf["journalist"] = cont[2]
        inf["img_coment"], inf["content"], inf["journalist"] = get_read_content(news_url)

        inf["news_url"] = news_url
        img_url = inf["img_url"] = head.find("a").find("img")

        if img_url is not None:
            inf["img_url"] = img_url["src"][:-14]
            inf["title"] = img_url["alt"]
        else:
            inf["title"] = head.text.strip()

        dd = news.find("dl").find("dd")
        inf["lede"] = dd.find("span",{"class":"lede"}).text
        inf["writing"] = dd.find("span",{"class":"writing"}).text

        try:
            news_page_df = news_page_df.append(inf,ignore_index=True)
        except:
            news_page_df = pd.DataFrame([inf])
        time.sleep(random.randrange(5,15))
    return news_page_df

In [13]:
news_list = list_page.find("div", {"class": "content"})
news_list = news_list.find_all("li")

con = get_content(news_list)
con

,date,title,lede,writing,journalist,news_url,img_url,content,img_coment
0,20211109,"""매운급식 인권침해""…정치하는엄마들, 병설유치원 인권위 진정",(서울=연합뉴스) 송은경 기자 = 시민단체 '정치하는엄마들'은 매운 급식을 제공하는...,연합뉴스,송은경 기자,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/001/2...,(서울=연합뉴스) 송은경 기자 = 시민단체 '정치하는엄마들'은 매운 급식을 ...,'매운 급식' 제보 사례[정치하는엄마들 제공. 재판매 및 DB 금지]


In [64]:
if "next" in list_page.find("div",{"class":"paging"}).find_all("a")[-1]["class"]:
    news_list_url_list = list_page.find("div",{"class":"paging"}).find_all("a")[:-1]
    next_page_url = list_page.find("div",{"class":"paging"}).find_all("a")[-1]["href"]
else:
    news_list_url_list = list_page.find("div",{"class":"paging"}).find_all("a")
    next_page_url = None

In [63]:
for news_list_url in news_list_url_list:
    print(list_url_base+news_list_url["href"])

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=2
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=3
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=4
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=5
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=6
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=7
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=8
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=9
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=250&sid1=102&date=20211109&page=10


In [72]:
all_news_df = pd.DataFrame(
    columns=[
        "date",
        "title",
        "lede",
        "writing",
        "journalist",
        "news_url",
        "img_url",
        "content",
        "img_coment",
    ]
)
all_news_df


,date,title,lede,writing,journalist,news_url,img_url,content,img_coment
